In [6]:
from PIL import Image
import os

# Set the input and output directories
output_dir = 'sliced'

# mkdir if not exists
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Load the input image
image = Image.open(r"C:\Users\flori\Downloads\media_images_val_examples_2368_f81bed81c4f499c368b1.png")

# Get the width and height of the input image
width, height = image.size

# Ensure the height is a multiple of 224
assert height % 224 == 0, "Image height must be a multiple of 224"

# Slice the image into individual 224x224 PNGs
for y in range(0, height, 224):
    for x in range(0, width, 224):
        crop_box = (x, y, x + 224, y + 224)
        crop_image = image.crop(crop_box)

        # Replace black pixels with white
        crop_image = crop_image.convert("RGBA")
        data = crop_image.getdata()
        new_data = [(230, 230, 230, a) if r == 0 and g == 0 and b == 0 else (r, g, b, a) for r, g, b, a in data]
        crop_image.putdata(new_data)

        # Upscale the cropped image by 16 using no interpolation
        upscaled_image = crop_image.resize((3584, 3584), resample=Image.NEAREST)

        # Save the upscaled image
        output_filename = f"output_image_{x}_{y}.png"
        upscaled_image.save(os.path.join(output_dir, output_filename))

In [1]:
import os
import rasterio
from rasterio.windows import Window
from PIL import Image

# Set the input and output directories
output_dir = 'output'

# Create the output directory if it doesn't exist
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Open the GeoTIFF file
with rasterio.open(r'C:\Users\flori\Downloads\TCI (6).tif') as src:
    # Get the image dimensions
    width, height = src.width, src.height

    # Slice the image into 224x224 chunks
    for i in range(0, width, 224):
        for j in range(0, height, 224):
            # Define the window for the current chunk
            window = Window(i, j, min(224, width - i), min(224, height - j))

            # Read the data for the current chunk
            data = src.read(window=window)

            # Create a PIL image from the data
            image = Image.fromarray(data.transpose(1, 2, 0))

            # Save the image as a PNG file
            output_file = os.path.join(output_dir, f'chunk_{i}_{j}.png')
            image.save(output_file)
            print(f'Saved {output_file}')

Saved output\chunk_0_0.png
Saved output\chunk_0_224.png
Saved output\chunk_0_448.png
Saved output\chunk_0_672.png
Saved output\chunk_0_896.png
Saved output\chunk_0_1120.png
Saved output\chunk_0_1344.png
Saved output\chunk_0_1568.png
Saved output\chunk_0_1792.png
Saved output\chunk_0_2016.png
Saved output\chunk_0_2240.png
Saved output\chunk_0_2464.png
Saved output\chunk_0_2688.png
Saved output\chunk_0_2912.png
Saved output\chunk_0_3136.png
Saved output\chunk_0_3360.png
Saved output\chunk_0_3584.png
Saved output\chunk_0_3808.png
Saved output\chunk_0_4032.png
Saved output\chunk_0_4256.png
Saved output\chunk_0_4480.png
Saved output\chunk_0_4704.png
Saved output\chunk_0_4928.png
Saved output\chunk_0_5152.png
Saved output\chunk_0_5376.png
Saved output\chunk_0_5600.png
Saved output\chunk_0_5824.png
Saved output\chunk_0_6048.png
Saved output\chunk_0_6272.png
Saved output\chunk_0_6496.png
Saved output\chunk_0_6720.png
Saved output\chunk_0_6944.png
Saved output\chunk_0_7168.png
Saved output\chun

In [7]:
import os
import rasterio
import numpy as np
from PIL import Image

# Set the path to the GeoTiff file
geotiff_path = '../data/stacked_features.tif'

# Open the GeoTiff file
with rasterio.open(geotiff_path) as src:
    # Get the number of bands (layers) in the GeoTiff
    num_bands = src.count

    # Assuming 9 timesteps with 6 layers each, where the first 3 are RGB
    num_timesteps = 9
    num_layers_per_timestep = 6
    num_rgb_layers = 3

    # Create the output directory if it doesn't exist
    output_dir = 'output_directory'
    os.makedirs(output_dir, exist_ok=True)

    # Loop through the timesteps and extract the RGB layers
    for timestep in range(num_timesteps):
        # Calculate the starting and ending band indices for the current timestep
        start_band = timestep * num_layers_per_timestep
        end_band = start_band + num_rgb_layers

        # Read the RGB layers for the current timestep
        rgb_data = src.read([start_band + 1, start_band + 2, start_band + 3])

        # Transpose the data to have the shape (height, width, channels)
        rgb_data = np.transpose(rgb_data, (1, 2, 0))

        # Create a PIL Image object from the RGB data
        image = Image.fromarray(rgb_data.astype(np.uint8))

        # Save the image as a PNG file
        output_path = os.path.join(output_dir, f'timestep_{timestep+1}.png')
        image.save(output_path)
        print(f'Saved timestep {timestep+1} RGB layers as {output_path}')

print('Extraction complete!')

Saved timestep 1 RGB layers as output_directory\timestep_1.png
Saved timestep 2 RGB layers as output_directory\timestep_2.png
Saved timestep 3 RGB layers as output_directory\timestep_3.png


IndexError: band index 19 out of range (not in (1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18))